In [34]:
from threading import Thread
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)

from transformers import DataCollatorWithPadding
from typing import List, Dict, Any
from transformers import PreTrainedTokenizerBase

In [38]:
from datasets import load_dataset
ds = load_dataset("Hankbeasley/polycoder")






In [39]:
def preprocess_function(examples):
    parts = examples['accept'].split('<think>')
    if (len(parts) < 2 or len(parts) > 2):
        return None
    assistantparts = examples['accept'].split("<｜Assistant｜>")
    if (len(assistantparts) > 2):
        return None
    examples['prompt'] = parts[0]
    examples['chosen'] = parts[1]
    partsrejected = examples['reject'].split('<think>')
    examples['rejected'] = "<think>" + partsrejected[1]
    return examples

ds = ds['train'].map(preprocess_function)
#print(ds)
formated = ds.remove_columns(['accept', 'reject', 'testname'])
print(formated)

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 8938
})


In [42]:

#print(formated[0]['prompt'])
#print(formated)
parts = ds[0]['accept'].split('<think>')
prompt = ds[0]['accept'].split('<think>')[0]
#print(parts[1])

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
tokenized = tokenizer(formated[0]['rejected'], add_special_tokens=True)
print(tokenized["input_ids"])
#print(tokenizer.convert_ids_to_tokens(tokenized["input_ids"]))
#print(tokenizer.eos_token)


[151646, 151648, 198, 71486, 11, 358, 2776, 16601, 448, 21828, 458, 64595, 309, 24279, 24626, 1667, 12914, 13, 576, 5795, 374, 311, 3270, 264, 729, 429, 646, 1477, 279, 4396, 15723, 31172, 369, 1817, 6524, 1741, 429, 279, 2661, 23606, 9982, 830, 13, 6771, 752, 1438, 1495, 1246, 358, 646, 5486, 419, 382, 5338, 11, 358, 1184, 311, 3535, 279, 3491, 17247, 13, 1674, 759, 309, 24279, 46523, 8290, 1817, 6524, 448, 264, 4911, 15723, 11, 323, 279, 6388, 11931, 4157, 387, 7168, 13, 576, 3110, 2661, 374, 55663, 488, 18447, 284, 93462, 11, 892, 46918, 311, 220, 24, 20, 21, 22, 488, 220, 16, 15, 23, 20, 284, 220, 16, 15, 21, 20, 17, 13, 8886, 6524, 33210, 311, 264, 2155, 15723, 11, 323, 279, 2629, 1969, 387, 4396, 382, 4416, 11, 279, 729, 358, 2776, 311, 3270, 11, 11625, 1507, 3880, 311, 1896, 458, 23606, 438, 1946, 323, 470, 458, 1633, 12731, 1817, 6524, 311, 1181, 12159, 15723, 382, 40, 3278, 1191, 553, 22314, 279, 1946, 23606, 13, 576, 1946, 686, 387, 264, 914, 304, 279, 3561, 330, 61674, 488, 